In [ ]:
# NBVAL_SKIP
import os
#  os.environ['SPS_HOME'] = '/mnt/storage/annalena_data/sps_fsps'
#os.environ['SPS_HOME'] = '/home/annalena/sps_fsps'
#os.environ['SPS_HOME'] = '/Users/annalena/Documents/GitHub/fsps'
os.environ['SPS_HOME'] = '/export/home/aschaibl/fsps'
os.environ['CUDA_VISIBLE_DEVICES'] = '0, 1, 2, 3, 4 '


In [2]:
#NBVAL_SKIP
import matplotlib.pyplot as plt
from rubix.core.pipeline import RubixPipeline 
import os
config = {
    "pipeline":{"name": "calc_ifu"},
    
    "logger": {
        "log_level": "DEBUG",
        "log_file_path": None,
        "format": "%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    },
    "data": {
        "name": "IllustrisAPI",
        "args": {
            "api_key": os.environ.get("ILLUSTRIS_API_KEY"),
            "particle_type": ["stars"],
            "simulation": "TNG50-1",
            "snapshot": 99,
            "save_data_path": "data",
        },
        
        "load_galaxy_args": {
        "id": 14,
        "reuse": True,
        },
        
        "subset": {
            "use_subset": True,
            "subset_size": 400000,
        },
    },
    "simulation": {
        "name": "IllustrisTNG",
        "args": {
            "path": "data/galaxy-id-14.hdf5",
        },
    
    },
    "output_path": "output",

    "telescope":
        {"name": "MUSE",
         "psf": {"name": "gaussian", "size": 5, "sigma": 0.6},
         "lsf": {"sigma": 0.5},
         "noise": {"signal_to_noise": 100,"noise_distribution": "normal"},},
    "cosmology":
        {"name": "PLANCK15"},
        
    "galaxy":
        {"dist_z": 0.1,
         "rotation": {"type": "edge-on"},
        },
        
    "ssp": {
        "template": {
            "name": "FSPS"
        },
        "dust": {
                "extinction_model": "Cardelli89",
                "dust_to_gas_ratio": 0.01,
                "dust_to_metals_ratio": 0.4,
                "dust_grain_density": 3.5,
                "Rv": 3.1,
            },
    },        
}

2025-05-08 23:07:50,850 - rubix - INFO - 
   ___  __  _____  _____  __
  / _ \/ / / / _ )/  _/ |/_/
 / , _/ /_/ / _  |/ /_>  <
/_/|_|\____/____/___/_/|_|


2025-05-08 23:07:50,853 - rubix - INFO - Rubix version: 0.0.post417+g76e9abf.d20250424
2025-05-08 23:07:50,853 - rubix - INFO - JAX version: 0.6.0
2025-05-08 23:07:50,854 - rubix - INFO - Running on [CudaDevice(id=0), CudaDevice(id=1), CudaDevice(id=2), CudaDevice(id=3)] devices


In [3]:
import jax
import jax.numpy as jnp

n_particles = 400_000

age = jnp.linspace(0, 20, n_particles, )
metallicity = jnp.linspace(0., 0.05, n_particles, )

from jax.sharding import Mesh, PartitionSpec as P
from jax.experimental import shard_map
from jax.sharding import NamedSharding



devices = jax.devices()
mesh = Mesh(devices, axis_names=('N_particles',))
sharding = NamedSharding(mesh, P('N_particles')) 

age = jax.device_put(age, sharding)
metallicity = jax.device_put(metallicity, sharding)

In [4]:
age = jnp.atleast_1d(age)
metallicity = jnp.atleast_1d(metallicity)

In [7]:
from rubix.core.ssp import get_lookup_interpolation

In [8]:
lookup_interpolation = get_lookup_interpolation(config)

2025-05-08 23:08:16,661 - rubix - DEBUG - Method not defined, using default method: cubic


In [9]:
print("lookup_interpolation", lookup_interpolation)

lookup_interpolation Partial(<PjitFunction of <function interp2d at 0x730aa0d6a8e0>>, method='cubic', x=Array([4.4904351e-05, 1.4200003e-04, 2.5251572e-04, 4.4904352e-04,
       7.9852482e-04, 1.4200003e-03, 2.5251573e-03, 4.4904351e-03,
       7.9852482e-03, 1.4199999e-02, 2.5251566e-02, 4.4904340e-02],      dtype=float32), y=Array([1.00000005e-04, 1.12201837e-04, 1.25892519e-04, 1.41253782e-04,
       1.58489333e-04, 1.77827940e-04, 1.99526214e-04, 2.23872063e-04,
       2.51188700e-04, 2.81838322e-04, 3.16227757e-04, 3.54813354e-04,
       3.98107077e-04, 4.46683698e-04, 5.01187285e-04, 5.62341302e-04,
       6.30957307e-04, 7.07945612e-04, 7.94328400e-04, 8.91251024e-04,
       1.00000005e-03, 1.12201832e-03, 1.25892507e-03, 1.41253788e-03,
       1.58489333e-03, 1.77827943e-03, 1.99526199e-03, 2.23872066e-03,
       2.51188688e-03, 2.81838328e-03, 3.16227763e-03, 3.54813342e-03,
       3.98107106e-03, 4.46683681e-03, 5.01187285e-03, 5.62341325e-03,
       6.30957261e-03, 7.0794564

In [10]:
def lookup_interpolation_lax(age_metallicity):
    age, metallicity = age_metallicity
    return lookup_interpolation(age, metallicity)

interpolation = jax.lax.map(lookup_interpolation_lax, (age, metallicity), batch_size=1)

2025-05-08 23:08:20.694754: E external/xla/xla/service/gpu/gpu_hlo_schedule.cc:652] The byte size of input/output arguments (9621985660) exceeds the base limit (8654290944). This indicates an error in the calculation!
2025-05-08 23:08:20.703903: W external/xla/xla/hlo/transforms/simplifiers/hlo_rematerialization.cc:3021] Can't reduce memory use below 0B (0 bytes) by rematerialization; only reduced to 8.94GiB (9597600124 bytes), down from 8.94GiB (9597600124 bytes) originally
2025-05-08 23:08:31.169703: W external/xla/xla/tsl/framework/bfc_allocator.cc:501] Allocator (GPU_0_bfc) ran out of memory trying to allocate 8.93GiB (rounded to 9590400000)requested by op 
2025-05-08 23:08:31.169909: W external/xla/xla/tsl/framework/bfc_allocator.cc:512] **__________________________________________________________________________________________________
E0508 23:08:31.169937 2740412 pjrt_stream_executor_client.cc:2839] Execution of replica 0 failed: RESOURCE_EXHAUSTED: Out of memory while trying t

XlaRuntimeError: RESOURCE_EXHAUSTED: Out of memory while trying to allocate 9590400000 bytes.: while running replica 0 and partition 0 of a replicated computation (other replicas may have failed as well).

In [4]:
_, interpolation = jax.lax.scan(
                    lambda carry, x: (carry, lookup_interpolation_lax(x)),
                    None,
                    (age, metallicity),
                )

NameError: name 'lookup_interpolation_lax' is not defined

[CudaDevice(id=0), CudaDevice(id=1)]